In [ ]:
import pandas as pd

In [ ]:
x_train_path = "data/X_train_Hi5.csv"
X_train = pd.read_csv(x_train_path)

In [ ]:
x_test_path = "data/X_test_Hi5.csv"
X_test = pd.read_csv(x_test_path)

In [ ]:
X_train.head()

## Filling the NaNs

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


# Calculate percentage of NaN values for each column
nan_percentage = (X_train.isna().mean() * 100)

plt.figure(figsize=(15, 8))  # Increase figure size for readability
nan_percentage.sort_values(ascending=False).plot(kind='bar')  # Sort for better visualization
plt.title('Percentage of NaN Values per Column')
plt.ylabel('Percentage (%)')
plt.xlabel('Columns')
plt.xticks(rotation=90, fontsize=8)  # Rotate and adjust font size for readability
plt.tight_layout()
plt.show()

In [ ]:
X_train["piezo_station_commune_code_insee"][X_train["insee_%_agri"].isna()].unique()

In [ ]:
df = X_train[X_train["piezo_station_commune_code_insee"].isin(X_train["piezo_station_commune_code_insee"][X_train["insee_%_agri"].isna()].unique())][["insee_%_agri"]].isna().groupby(X_train["piezo_station_commune_code_insee"])

In [ ]:
X_train[X_train["piezo_station_commune_code_insee"].str.match(r"^(16)\d{3}")]

In [ ]:
def nans_per_col(nan_col, group_col):
    print(X_train[X_train[nan_col].isin(X_train[group_col][X_train[nan_col].isna()].unique())][[nan_col]].isna().groupby(X_train[group_col]))

In [ ]:
df.agg("mean")

In [ ]:
print_full(pd.to_numeric(X_train["insee_%_agri"].dropna(), errors="coerce").groupby(X_train["piezo_station_department_code"][X_train["piezo_station_department_code"].isin(X_train["piezo_station_department_code"][X_train["insee_%_agri"].isna()].unique())]).agg("mean"))

In [ ]:
X_train.iloc[140]["insee_%_agri"]

In [ ]:
X_train["insee_%_agri"][~X_train["insee_%_agri"].apply(lambda x: isinstance(x, (int, float)))]

In [ ]:
X_train["meteo_if_thunderstorm"].unique()

Ouch, on va plutôt traiter les colonnes avec peu de NaNs dans un premier temps

## Correlation matrix meteo

In [ ]:
boolean_columns = [
    "meteo_if_fog",
    "meteo_if_mist",
    "meteo_if_lightning",
    "meteo_if_smoke",
    "meteo_if_frost",
    "meteo_if_hail",
    "meteo_if_sleet",
    "meteo_if_snow",
    "meteo_if_thunderstorm",
    "meteo_if_dew",
    "meteo_if_snow_ground",
    "meteo_if_black_ice"
]

In [ ]:
print_full(X_train[boolean_columns].isna().mean().sort_values() * 100)

In [ ]:
print_full(X_train[boolean_columns][X_train[boolean_columns] == 1].count().sort_values() / X_train[boolean_columns].count())

In [ ]:
X_train[["meteo_snow_thickness_6h", "meteo_snow_thickness_max"]].describe()

In [ ]:
X_train["meteo_snow_thickness_6h"].corr(X_train["meteo_snow_thickness_max"])

In [ ]:
df = X_train.copy()
df["na_%"] = df.isna().mean().sort_values() * 100

In [ ]:
print_full(X_train.isna().mean().sort_values() * 100)

In [ ]:
replace_with_mean = ["meteo_rain_height", "insee_%_agri", ]

In [ ]:
X_train["meteo_rain_height"].describe()

In [ ]:
print_full(X_train.head())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assume 'df' is the DataFrame containing all the meteo columns
df = X_train.copy()
df["piezo_groundwater_level_category"].replace({"Very Low": 0, "Low": 1, "Average": 2, "High": 3, "Very High": 4}, inplace=True)

# Select only numeric meteo columns
numeric_meteo_columns = [
    "meteo_cloudiness_height", "meteo_frost_duration", "meteo_wetting_duration",
    "meteo_humidity_duration_below_40%", "meteo_humidity_duration_above_80%",
    "meteo_radiation_direct", "meteo_wind_direction_max_inst",
    "meteo_wind_direction_max_inst_2m", "meteo_wind_direction_max_avg",
    "meteo_evapotranspiration_grid", "meteo_evapotranspiration_Monteith",
    "meteo_wind_speed_avg_2m", "meteo_wind_speed_avg_10m", "meteo_wind_max",
    "meteo_wind_max_2m", "meteo_wind_max_3s", "meteo_wind_avg", "meteo_radiation",
    "meteo_snow_height", "meteo_radiation_IR", "meteo_sunshine_duration",
    "meteo_cloudiness", "meteo_snow_thickness_6h", "meteo_snow_thickness_max",
    "meteo_pressure_avg", "meteo_pression_maxi", "meteo_rain_height",
    "meteo_sunshine_%", "meteo_amplitude_tn_tx", "meteo_temperature_avg",
    "meteo_temperature_avg_threshold", "meteo_temperature_min",
    "meteo_temperature_min_50cm", "meteo_temperature_min_ground",
    "meteo_temperature_avg_tntm", "meteo__pressure_saturation_avg",
    "meteo_temperature_max", "meteo_humidity_avg", "meteo_humidity_min",
    "meteo_radiation_UV", "meteo_radiation_UV_max", "meteo_humidity_max",
    "piezo_groundwater_level_category"
]

# Filter the DataFrame for the selected numeric columns
numeric_data = df[numeric_meteo_columns].apply(pd.to_numeric, errors="coerce")

# Compute the correlation matrix
correlation_matrix = numeric_data.corr()

# Plot the heatmap
plt.figure(figsize=(15, 10))
sns.heatmap(correlation_matrix, annot=False, cmap="coolwarm", fmt=".2f", cbar=True)
plt.title("Correlation Matrix of Meteorological Variables")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## Remplacement des NaNs avec sklearn

In [ ]:
from sklearn.impute import SimpleImputer 

In [ ]:
remplace_with_mean_cols = []

## Exploration

In [ ]:
X_train["piezo_measurement_date"].sort_values()

In [ ]:
X_train["piezo_station_bss_id"].nunique()

In [ ]:
len(X_train[X_train["piezo_station_bss_id"] == "BSS001TTQQ"])

In [ ]:
print_full(X_train.columns)

In [ ]:
print_full(X_train[X_train["piezo_station_bss_id"] == "BSS001TTQQ"])

### plotting des dates

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
date_range = pd.date_range(start='2020-01-01', end='2023-09-30', freq='D')

In [ ]:
df_train = X_train.copy()
df_train['present'] = 1
df_train['piezo_measurement_date'] = pd.to_datetime(df_train['piezo_measurement_date']).dt.normalize()
pivot_df_train = df_train.pivot_table(index='piezo_measurement_date', columns='piezo_station_bss_id', values='present', fill_value=0)
pivot_df_train = pivot_df_train.reindex(date_range, fill_value=0)

In [ ]:
df_test = X_test.copy()
df_test['present'] = 1
df_test['piezo_measurement_date'] = pd.to_datetime(df_test['piezo_measurement_date']).dt.normalize()
pivot_df_test = df_test.pivot_table(index='piezo_measurement_date', columns='piezo_station_bss_id', values='present', fill_value=0)
pivot_df_test = pivot_df_test.reindex(date_range, fill_value=0)

In [ ]:
df = pivot_df_train - pivot_df_test

In [ ]:
plt.figure(figsize=(20, 10))
sns.heatmap(df.T, cmap=['orange', 'white', 'blue'], cbar=True)
plt.xlabel('Date')
plt.ylabel('Piezo Station BSS ID')
plt.title('Measurement Presence Heatmap')
plt.show()

In [ ]:
df_test['piezo_measurement_date'].dtypes

In [ ]:
df_test['month'] = df_test['piezo_measurement_date'].dt.to_period('M')
df_test["month"].unique()

## Dates sur X_test

In [ ]:
x_test_path = "data/X_test_Hi5.csv"
X_test = pd.read_csv(x_test_path)

In [ ]:
len(X_test)

In [ ]:
X_test["piezo_measurement_date"].sort_values()

In [ ]:
X_test["piezo_measurement_date"].sort_values().hist()

In [ ]:
X_train["piezo_station_bss_id"].nunique()

## Other sheniningans

In [ ]:
X_train.rename(columns={"piezo_groundwater_level_category": "target"}, inplace=True)

In [ ]:
X_train["target"].unique()

In [ ]:
X_train[X_train["target"].isna()]

## Corrélation entre les features et la target

Est-ce que ça fait même sens de faire ça vu que la target est catégorielle ?

In [ ]:
from pandas import factorize
import numpy as np

In [ ]:
numerical_cols = X_train.select_dtypes(include=np.number)

In [ ]:
def print_full(x):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.4f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

In [ ]:
def corrwith(df, label):
    df["target_labels"] = df["target"].apply(lambda x: 1 if x == label else 0)
    print_full(abs(numerical_cols.corrwith(df["target_labels"])).sort_values())

In [ ]:
corrwith(X_train, ["target"])